In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os
from collections import Counter
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesRegressor

import random

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score,mean_absolute_error
from collections import Counter
from imblearn.over_sampling import SMOTE


## Dati

In [3]:
best5_CMI_fulldf_train = pd.read_csv("/Users/paolo/Downloads/esempi_droughts/best5_CMI_fulldf_train.csv")
best5_CMI_fulldf_val = pd.read_csv("/Users/paolo/Downloads/esempi_droughts/best5_CMI_fulldf_val.csv")
best5_CMI_fulldf_test = pd.read_csv("/Users/paolo/Downloads/esempi_droughts/best5_CMI_fulldf_test.csv")
targets_df_train = pd.read_csv("/Users/paolo/Downloads/esempi_droughts/targets_df_train.csv")
targets_df_val = pd.read_csv("/Users/paolo/Downloads/esempi_droughts/targets_df_val.csv")
targets_df_test = pd.read_csv("/Users/paolo/Downloads/esempi_droughts/targets_df_test.csv")


In [4]:
best5_CMI_fulldf_train

,Unnamed: 0,Adda_cyclostationary_mean_tg_3,Adda_cyclostationary_mean_rr_8w_0,Adda_cyclostationary_mean_rr_1w_0,Dora_cyclostationary_mean_tg_0,Dora_cyclostationary_mean_tg_12w_0,Dora_cyclostationary_mean_tg_12w_1,Dora_cyclostationary_mean_rr_12w_0,Dora_cyclostationary_mean_tg_24w_0,Emiliani1_cyclostationary_mean_rr_4w_1,...,Piemonte_Nord_cyclostationary_mean_tg_5,Piemonte_Nord_cyclostationary_mean_rr_4w_1,Piemonte_Sud_cyclostationary_mean_tg_0,Piemonte_Sud_cyclostationary_mean_rr_24w_0,Piemonte_Sud_cyclostationary_mean_rr_8w_1,Piemonte_Sud_cyclostationary_mean_rr_16w_1,Piemonte_Sud_cyclostationary_mean_tg_1w_0,Ticino_cyclostationary_mean_tg_0,Ticino_cyclostationary_mean_rr_4w_0,Ticino_cyclostationary_mean_tg_2
0,0,0.723001,1.814900,1.002579,-0.660721,-1.687475,-0.852681,-0.273561,-2.048114,2.175159,...,-0.787022,1.093488,-0.451234,3.106019,1.565205,3.762580,-0.480168,-0.410998,0.611605,-0.447071
1,1,1.550127,3.486901,1.976372,0.414456,-0.244265,0.435183,1.339306,-0.206067,1.433193,...,0.262022,1.461017,0.882324,3.786747,2.140184,4.338552,0.335774,0.803930,1.691336,0.647375
2,2,0.089708,2.176553,1.351690,-1.224984,-1.443935,-0.318581,0.336223,-1.642723,1.339315,...,-1.333074,0.877684,-0.830092,2.097704,1.168303,2.395103,0.130577,-0.911268,0.832271,-0.540914
3,3,1.303410,2.160372,0.465520,0.137561,-0.933379,-0.128341,0.580213,-1.039207,0.826347,...,0.261690,0.884840,0.865126,1.990877,1.116412,2.809543,0.099606,0.538884,0.859041,0.453677
4,4,0.581894,1.585804,0.405690,-0.950332,-1.292032,-0.243374,0.449412,-1.479871,1.856047,...,-1.357486,0.553921,-0.244129,1.294920,0.711381,1.953300,0.472017,-0.689709,0.647203,0.012920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,406,1.217855,-0.896405,-1.209534,1.153714,1.146649,0.886961,-0.924023,0.918164,-0.110660,...,1.342323,-0.740513,1.372812,-0.779095,-0.477228,-0.147719,1.357565,1.423525,-1.023397,1.938235
407,407,0.535153,-0.574062,0.827442,-0.262167,0.806876,0.630671,-0.699693,0.856020,-1.114120,...,-0.292368,-0.112187,-0.113460,-0.493034,-0.450756,0.423013,0.739625,0.065613,-0.154876,0.983134
408,408,0.270177,-0.362938,1.240716,0.761686,0.848483,0.530520,-0.678182,0.747310,-1.140167,...,0.666602,0.256594,0.642464,-0.406663,-0.241398,0.424600,0.310534,0.880453,0.083449,0.717246
409,409,-1.422653,-0.290838,-0.157669,-2.729620,0.231546,0.047550,-0.645966,0.409265,-1.091788,...,-2.956863,0.005699,-2.244103,-0.321048,-0.351044,0.314630,-0.965050,-2.448215,0.001375,-0.986677


In [5]:
targets_df_train

,Unnamed: 0,Adda,Dora,Emiliani1,Emiliani2,Garda_Mincio,Lambro_Olona,Oglio_Iseo,Piemonte_Nord,Piemonte_Sud,Ticino
0,0,-2.546951,-2.129508,-0.382765,-1.339879,-1.996014,-0.439541,-1.223671,-1.146332,-0.967137,-1.060146
1,1,-0.277191,-0.927136,0.319215,0.402993,0.498869,-0.019547,-0.087252,0.371173,0.070382,-0.405065
2,2,-0.534156,-0.555958,0.548542,0.282703,-0.428613,0.269293,-0.278268,0.379474,0.342478,0.125603
3,3,-0.666789,-0.718282,-0.010351,-0.030490,-0.583432,-0.434406,-0.774846,-0.099118,-0.442927,-0.514438
4,4,-0.447894,-1.008995,0.402030,0.451097,0.075938,-0.421060,0.169616,-0.292244,-0.019192,-0.625540
...,...,...,...,...,...,...,...,...,...,...,...
406,406,-0.263306,-0.784269,-0.060454,0.091910,0.032299,-0.212272,0.174854,-0.549184,-0.021388,-0.563495
407,407,-0.082282,-0.701139,0.761079,0.283224,0.132804,-0.299172,0.059327,-0.292423,-0.086330,-0.291984
408,408,0.331204,-0.655289,0.649694,0.847138,-0.214182,0.790614,0.748131,0.454529,0.576347,0.770822
409,409,-0.648940,-1.586325,-0.957702,0.552758,-2.409204,0.109054,-0.819305,-0.383480,0.146632,-0.412164


## MTL

In [32]:
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test,mod='linear'):

    # prendo tutte le colonne che iniziano con il nome di uno dei clusters
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    # replico tutte le righe per i tre df di train, val, test concatenando one hot encoding sulle colonne

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    if mod == 'RF':
        model_ohe = ExtraTreesRegressor()
    elif mod == 'SVR':
        model_ohe = SVR()
    elif mod == 'MLP':
        model_ohe = MLPRegressor(hidden_layer_sizes=5)
    else:
        model_ohe = LinearRegression()
        
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        print(mod+'\n')

        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(r2_score(targets_df_test[basin].values.ravel(), res))
        print(np.corrcoef(targets_df_test[basin].values.ravel(), res)[0][1])
        print(mean_absolute_error(targets_df_test[basin].values.ravel(), res))
        print('\n')

   # li ritorno solo per mostrare un esempio 
    return clusterdf_test_withClass,targets_df_test_unfolded


In [33]:
# esempio

clusterdf_test_withClass,targets_df_test_unfolded = MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)


Emiliani1
linear

0.32984044510617994
0.6094126344358477
0.5538016694325365


Emiliani2
linear

0.29261821856191017
0.5706274306097973
0.5587204729215168


Garda_Mincio
linear

0.21541914858760047
0.5706274306097974
0.6475309180936673




In [34]:
clusterdf_test_withClass

,Emiliani1_cyclostationary_mean_rr_4w_1,Emiliani1_cyclostationary_mean_tg_0,Emiliani1_cyclostationary_mean_rr_1w_16,Emiliani1_cyclostationary_mean_rr_24w_2,Emiliani2_cyclostationary_mean_tg_0,Emiliani2_cyclostationary_mean_rr_8w_0,Emiliani2_cyclostationary_mean_tg_4w_0,Emiliani2_cyclostationary_mean_rr_4w_5,Emiliani2_cyclostationary_mean_rr_8w_3,Garda_Mincio_cyclostationary_mean_rr_4w_2,Garda_Mincio_cyclostationary_mean_tg_1w_0,Garda_Mincio_cyclostationary_mean_rr_4w_1,Emiliani1,Emiliani2,Garda_Mincio
0,0.352378,-0.943170,-0.934859,1.267477,-0.823329,1.188554,1.521630,0.424887,0.208269,0.042597,0.249205,0.226752,1,0,0
1,0.228490,1.008992,-1.072484,1.170356,1.518898,1.631268,1.546902,0.189369,0.678054,0.100205,-0.110948,0.080919,1,0,0
2,-0.565427,1.713287,-0.042526,0.962091,1.351770,1.454169,1.513190,-0.247037,0.337938,-0.662670,1.180650,-0.513590,1,0,0
3,-0.653729,0.621095,0.044566,0.854003,0.555240,-0.051761,1.233871,-0.268305,-0.617941,-0.483908,1.440551,-0.369692,1,0,0
4,-0.104724,-0.484844,0.374816,0.891574,-0.553753,0.016142,0.610897,-0.143014,-0.435534,-0.108686,0.431518,-0.282093,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,0.721980,0.541911,2.060772,0.890943,0.782310,1.601112,1.900098,2.458964,-0.577633,2.228233,0.179840,2.545958,0,0,1
224,0.693338,1.483742,1.504021,0.202589,1.919141,2.164934,1.751234,2.902058,-0.309978,2.345024,1.304742,3.019941,0,0,1
225,1.205597,0.544086,-0.575270,-0.066425,1.144042,2.241536,1.404626,3.373420,-0.499985,2.975361,1.162720,3.435264,0,0,1
226,1.152843,0.202232,-1.321619,-0.318737,1.004464,2.286236,1.434069,3.278008,-0.520052,3.124628,0.531253,3.454292,0,0,1


In [35]:
targets_df_test_unfolded

,0
0,-0.470723
1,0.244241
2,-0.765885
3,-0.136111
4,-0.082400
...,...
679,2.290788
680,0.672729
681,-0.100217
682,0.798153


### Chiamata completa

In [36]:
for clust_basins in [['Emiliani1'],['Emiliani2'],['Garda_Mincio'],['Emiliani1','Emiliani2','Garda_Mincio'],['Adda'],['Ticino'],['Lambro_Olona'],['Oglio_Iseo'],['Adda','Lambro_Olona','Oglio_Iseo','Ticino'],['Piemonte_Nord'],['Piemonte_Sud'],['Dora'],['Piemonte_Nord','Piemonte_Sud','Dora']]:
    for model in ['linear','RF','MLP','SVR']:
        MTL_scores(mod=model, clust_basins=clust_basins, df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)


Emiliani1
linear

0.2860711267941549
0.5866027290374541
0.5642123216063224


Emiliani1
RF

0.13927650557371796
0.5267781613313297
0.6215263770721289


Emiliani1
MLP

0.24180031465634444
0.5822433618131628
0.5833968334854664


Emiliani1
SVR

0.19901893414411376
0.5584581499257306
0.5997702609947833


Emiliani2
linear

0.24353431838283113
0.5344501166880028
0.5806412006089803




/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Emiliani2
RF

0.13847538186589015
0.4467569752795254
0.6304475291957246


Emiliani2
MLP

0.23714840023616768
0.5389477946291944
0.590249939557685


Emiliani2
SVR

0.036537057922730876
0.39855913999715614
0.652566915943466


Garda_Mincio
linear

0.1713073084951655
0.5308526726022612
0.6673751457293776


Garda_Mincio
RF

-0.17586527307148425
0.3326804850386845
0.7982707159777831


Garda_Mincio
MLP

0.11719745742876708
0.5026089717741407
0.6945716589485219


Garda_Mincio
SVR

0.11639479418853926
0.49296693415787174
0.6918328146383471


Emiliani1
linear

0.32984044510617994
0.6094126344358477
0.5538016694325365


Emiliani2
linear

0.29261821856191017
0.5706274306097973
0.5587204729215168


Garda_Mincio
linear

0.21541914858760047
0.5706274306097974
0.6475309180936673


Emiliani1
RF

0.27594582022259573
0.581221965076638
0.5768048192313071


Emiliani2
RF

0.2592029045876286
0.536076013165696
0.5807657984279991


Garda_Mincio
RF

0.1458938452529257
0.5243759819657985
0.6892454739081382




/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Emiliani1
MLP

0.2783789849431001
0.5917683254230007
0.559791417549203


Emiliani2
MLP

0.15824217172030108
0.5115246642084275
0.6036755090503201


Garda_Mincio
MLP

0.1166726446888735
0.5060712134503316
0.6727844409300827


Emiliani1
SVR

0.22756831996161075
0.5338850776830397
0.5956442012317194


Emiliani2
SVR

0.24694517494386437
0.5352507176294962
0.5755589055260077


Garda_Mincio
SVR

0.16086383980811692
0.5411133778415242
0.6685509365092357


Adda
linear

0.09761785079534946
0.49523742944289256
0.6475554383978339


Adda
RF

-0.15470518824879953
0.3059653701216688
0.7262861080356718


Adda
MLP

0.12888995534969705
0.5335173437230976
0.6372854704845325


Adda
SVR

0.14656012970400534
0.48409069447712055
0.630048088555332


Ticino
linear

0.15480738872169852
0.450722272590247
0.6155108450998603


Ticino
RF

0.09683590701368272
0.41259497469694945
0.6495183253273923




...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Adda
MLP

0.08872666192398382
0.3963763668095657
0.6463530742244025


Lambro_Olona
MLP

0.140224617429338
0.42644847362605287
0.6357163538879874


Oglio_Iseo
MLP

0.10165610879119202
0.3927450649214135
0.6418338018865152


Ticino
MLP

0.12810285598622895
0.4129099653740683
0.6293960417289944


Adda
SVR

0.13625220819369055
0.46099544542827825
0.6276527682952157


Lambro_Olona
SVR

0.1780207203157389
0.4763163317457565
0.6237851752662269


Oglio_Iseo
SVR

0.11564756559460576
0.4249307698025788
0.6320879842063996


Ticino
SVR

0.1473444661268214
0.4506563682528424
0.6172539127377966


Piemonte_Nord
linear

0.06294698049742187
0.3197366424306478
0.6811750372325718


Piemonte_Nord
RF

0.0018150223481597783
0.32516081325805857
0.7076757470599346


Piemonte_Nord
MLP

0.05557878295635632
0.31869410594555375
0.680338429337515


Piemonte_Nord
SVR

0.100900398423167
0.3658920011956584
0.6734832877857422


Piemonte_Sud
linear

0.12281091488518436
0.38067192585662524
0.6761272691916316


Piemonte_

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Piemonte_Sud
SVR

0.2574534653089038
0.5248399588088687
0.6308620895789271


Dora
linear

-0.20039422688158504
0.002311618513049107
0.7185940604358516


Dora
RF

-0.11131976190516624
0.2702248318515495
0.6817823106979451


Dora
MLP

-0.1355988914383892
-0.00866618452753487
0.7120948653536924


Dora
SVR

-0.20600240823981997
0.26758819138111123
0.7180144917551289


Piemonte_Nord
linear

0.03010554429615042
0.27140251864854387
0.7008773712186257


Piemonte_Sud
linear

-0.08690395319182054
0.13914869895263993
0.7414493978678711


Dora
linear

-0.12902325803634618
0.12203429578401123
0.6978143815912238


Piemonte_Nord
RF

0.1500778637916801
0.42786370598877577
0.64632716159485


Piemonte_Sud
RF

0.16366427706194986
0.42577260923724586
0.6699775256642209


Dora
RF

0.1358142705140325
0.45761506754701586
0.5987338652743021


Piemonte_Nord
MLP

0.09615747780982897
0.38624572640871724
0.6830832150243638


Piemonte_Sud
MLP

0.13436958128742782
0.40322398426234135
0.6806764323941131


Dora
MLP



/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [4]:
import pandas as pd
import geopandas as gpd

p = pd.read_pickle("/Users/paolo/Downloads/thesis-master/data/spatial_aggregation_components/basins_variable_components.pickle")
p["Ticino"]["cyclostationary_mean_tg"]     

,geometry,component
0,POINT (8.14986 45.94986),0
1,POINT (8.24986 45.94986),0
2,POINT (8.34986 46.04986),0
3,POINT (8.44986 46.04986),0
4,POINT (8.14986 46.04986),0
...,...,...
87,POINT (8.84986 45.34986),2
88,POINT (8.94986 45.34986),2
89,POINT (8.94986 45.24986),2
90,POINT (9.04986 45.24986),2


In [14]:
import xarray
xp = p["Ticino"]["cyclostationary_mean_tg"]
x = xp.geometry.x
y = xp.geometry.y
val = xp.component

In [17]:
df = pd.DataFrame()
df["x"] = x
df["y"] = y
df["val"] = val
df

,x,y,val
0,8.14986,45.949861,0
1,8.24986,45.949861,0
2,8.34986,46.049861,0
3,8.44986,46.049861,0
4,8.14986,46.049861,0
...,...,...,...
87,8.84986,45.349861,2
88,8.94986,45.349861,2
89,8.94986,45.249861,2
90,9.04986,45.249861,2


In [27]:
df.to_csv("ticino_savedForTiff.csv")

In [5]:
import pandas as pd
import rioxarray as xr

In [32]:
df = pd.read_csv("ticino_savedForTiff.csv")
df = df.iloc[:,1:]
da = df.set_index(['y', 'x']).to_xarray()

da = da.rio.set_crs('epsg:3035')
da.rio.to_raster("/Users/paolo/Downloads/ticino_trial.tiff")


In [33]:
da

<xarray.Dataset>
Dimensions:  (y: 14, x: 14)
Coordinates:
  * y        (y) float64 45.25 45.35 45.45 45.55 ... 46.25 46.35 46.45 46.55
  * x        (x) float64 7.95 8.05 8.15 8.25 8.35 ... 8.85 8.95 9.05 9.15 9.25
Data variables:
    val      (y, x) float64 nan nan nan nan nan nan ... 1.0 1.0 1.0 nan nan nan